In [ ]:
import pandas as pd
import numpy as np
import os
import re

#initial data obtained from dump.sql, syntax converted to sqlite for easy .sqlite export to .csv
#read in category tables data 
dftemp_A = pd.read_csv("C:\\datasources\\CategoryArticle.csv", sep='|')
dftemp_B = pd.read_csv("C:\\datasources\\Category.csv", sep='|', usecols=['cid','url'])

#combine key tables
d = pd.Series(dftemp_B.url.values,index=dftemp_B.cid).to_dict()
dftemp_A['tag'] = dftemp_A['cid'].map(d)

#create multiindex hierarchy
dftemp_A.set_index(['ArticleId','cid'], inplace=True)
dftemp_A.sort_index(inplace=True)

#transform dataframe to dictionary where an ArticleId input returns correct tags list
v = dftemp_A.groupby('ArticleId')['tag'].apply(lambda t: list(t)).to_dict()
#check it by inserting ArticleId for XXXX in "print(v[XXXX])" 

#create file for building category to store edges in graph
dftemp_A.to_csv("C:\\datasources\\tagedges.csv", sep='|', index=True, encoding='utf-8')

#read in raw articles table data
dftemp_DB = pd.read_csv("C:\\datasources\\Article.csv", sep='|', usecols=['ArticleId','Title','PublicationDate','Publication','Links','Description','Priority','url','url2'])

#make some unique wtk urls 
makeurl = dftemp_DB['url'].astype(str)
dftemp_DB['wtkURL'] = "https://www.wanttoknow.info/a-" + makeurl
  
#make tags column and populate with dictionary v
dftemp_DB['tags'] = dftemp_DB['ArticleId'].map(v)  

#check it with something like dftemp_DB.head(12)

#send to "|" delim .csv as hacky way to bypass jupyter notebook default io limits; restart kernel
dftemp_DB.to_csv("C:\\datasources\\WTKpreprocessed.csv", sep='|', index=False, encoding='utf-8')

In [ ]:
#begin again after restarting kernel to avoid exceeding jupyter notebook default io limits when testing
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import os
import re

#read in preprocessed file
dfP = pd.read_csv("C:\\datasources\\WTKpreprocessed.csv", sep='|', encoding='utf-8')

#split raw story into summary and note. 
dfP['Summary'], dfP['Note'] = dfP['Description'].str.split('Note:', 1).str 

#create unique tag columns to make values quickly mappable for graph generation
dfP['tags'] = dfP['tags'].astype(str).apply(lambda v: v.replace('\'', ''))
dfP.tags = dfP.tags.str[1:-1].str.split(',').tolist()
dfT = pd.DataFrame(dfP.tags.values.tolist(), dfP.index).add_prefix('tag_')

form = lambda f: 'tag_{}'.format(f + 1)
pd.DataFrame(
    dfP.tags.values.tolist(),
    dfP.index, dtype=object
).fillna('').rename(columns=form)

df = pd.concat([dfP, dfT], axis=1)

#add tagcount column for advanced indexing convenience
df['tagcount'] = df.tags.apply(lambda l: len(l))

#initialize some lists for data extraction and cleaning
Quotes = []
Summaries = []
Note_text = []
Adtl_ref_links = []
Note_ref_links = []
Media_sources = []

#set variables up for parsing markup
stories = df['Summary'].astype(str).values.tolist()
notes = df['Note'].astype(str).values.tolist()
medias = df['Publication'].astype(str).values.tolist()

#some loops to extract desired text and links
for i in stories:
    c = bs(i,'lxml')
    try:
        quote = c.strong.text
    except (AttributeError):
        quote = c.text.split('. ')[0]
    summary = c.text
    ref_links = []
    for n in c.find_all('a'):
        xlinks = n.get('href')
        ref_links.append(xlinks)
    Quotes.append(quote)
    Summaries.append(summary)
    Adtl_ref_links.append(ref_links)

#switch parser from lxml for notes parsing because different parsers are good for different things    
for i in notes:
    c = bs(i,'html.parser')
    note = c.text    
    note_links = []
    for n in c.find_all('a'):
        nlinks = n.get('href')
        note_links.append(nlinks)
    Note_text.append(note)
    Note_ref_links.append(note_links)

#initial cleanup of pub. bs4 282 User Warning may show up because some media sources are urls. no biggie    
for i in medias:
    c = bs(i,'html.parser')
    media = c.text
    Media_sources.append(media)
    
#make new dataframe from extracted data    
df3 = pd.DataFrame({'quote': Quotes,
                  'description': Summaries,
                  'note': Note_text,
                  'pub': Media_sources,   
                  'Summary_ref_links': Adtl_ref_links,
                  'Note_links': Note_ref_links})    

#join new dataframe to big raw dataframe
dfR = pd.concat([df, df3], axis=1)

dfR.drop(['Publication', 'Summary', 'url', 'url2'], axis=1, inplace=True)

#begin cleanup of media sources
dfR['pub'], dfR['pub2'] = dfR['pub'].str.split('/', 1).str
dfR['pub'], dfR['pubdetail'] = dfR['pub'].str.split('(', 1).str
dfR['pubdetail'] = dfR['pubdetail'].astype(str).apply(lambda u: u.strip(')'))
dfR['pub'] = dfR['pub'].str.strip()
dfR['pub'] = dfR['pub'].astype(str).apply(lambda u: u.strip('"'))
dfR['pub'] = dfR['pub'].astype(str).apply(lambda r: r.replace('\'',''))

#use dfR.pub.unique() with pd.set_option('display.max_colwidth', 1000) to view raw media sources list
#dfR['pub'].value_counts()

#combine variations of media source into single item

#one-off translations are easy, but should eventually be moved to mediacondense list
dfR['pub'] = dfR['pub'].astype(str).apply(lambda r: r.replace('Bloomberg News Service', 'Bloomberg'))
dfR['pub'] = dfR['pub'].astype(str).apply(lambda r: r.replace('Scientific American Blog', 'Scientific American'))
dfR['pub'] = dfR['pub'].astype(str).apply(lambda r: r.replace('The Sacramento Bee', 'Sacramento Bee'))
dfR['pub'] = dfR['pub'].astype(str).apply(lambda r: r.replace('Mother Jones Magazine', 'Mother Jones'))
dfR['pub'] = dfR['pub'].astype(str).apply(lambda r: r.replace('Sydney Mountain Herald', 'Sydney Morning Herald'))
dfR['pub'] = dfR['pub'].astype(str).apply(lambda r: r.replace('The Nation magazine', 'The Nation'))
dfR['pub'] = dfR['pub'].astype(str).apply(lambda r: r.replace('Chicago Sun-Times News Group', 'Chicago Sun-Times'))
dfR['pub'] = dfR['pub'].astype(str).apply(lambda r: r.replace('Wired magazine', 'Wired'))
dfR['pub'] = dfR['pub'].astype(str).apply(lambda r: r.replace('The New Yorker magazine', 'The New Yorker'))
dfR['pub'] = dfR['pub'].astype(str).apply(lambda r: r.replace('Fortune magazine', 'Fortune'))
dfR['pub'] = dfR['pub'].astype(str).apply(lambda r: r.replace('The Atlantic Monthly', 'The Atlantic'))
dfR['pub'] = dfR['pub'].astype(str).apply(lambda r: r.replace('Tikkun Magazine - March', 'Tikkun Magazine'))
dfR['pub'] = dfR['pub'].astype(str).apply(lambda r: r.replace('The Daily Mail', 'Daily Mail'))

dfR.pub.head(3)


In [ ]:
#'mediacondense' list of dictionaries is built for when there are more than 2 variations of a media source
#should eventually be moved into two adtl files, one for raw list storage and one for handling list interactions 
mediacondense = []
abc = ['ABC News Australia', 'ABC News blog', 'ABC6', 'abcnews.com', 'WCPO - Cincinnatis ABC Affiliate', 'ABC News', 'ABC News blogs', 'ABC News Blog', 'ABC News Good Morning America', 'ABC New', 'ABC News Nightline', 'ABC15', 'ABC News 20', 'abc4.com', 'ABC Action News', 'ABCs Arizona Affiliate','WXYZ - Detroits ABC News Affiliate']
mcat1 = "ABC"
m1 = dict.fromkeys(abc, mcat1)
mediacondense.append(m1)

ode = ['Ode Magazine, June 2005 Issue', 'Ode Magazine, July 2005 Issue', 'Ode magazine']
mcat2 = "Ode Magazine"
m2 = dict.fromkeys(ode, mcat2)
mediacondense.append(m2)

nbc = ['NBC Milwaukee Affiliate', 'NBC Chicago', 'NBC Miami', 'NBC Washington', 'NBC Los Angeles', 'NBC Oklahoma City', 'NBC News']
mcat3 = "NBC"
m3 = dict.fromkeys(nbc, mcat3)
mediacondense.append(m3)

vfr = ['Vanity Fair August 2006 Issue', 'Vanity Fair September 2005 Issue', 'Vanity Fair magazine']
mcat4 = "Vanity Fair"
m4 = dict.fromkeys(vfr, mcat4)
mediacondense.append(m4)

nyt = ['The New York Times', 'New York Times Blog']
mcat5 = "New York Times"
m5 = dict.fromkeys(nyt, mcat5)
mediacondense.append(m5)

unw = ['U.S. News and World Report', 'U.S. News & World Report', 'U.S. News & World Report blog', 'US News & World Report magazine', 'US News & World Report']
mcat6 = "US News and World Report"
m6 = dict.fromkeys(unw, mcat6)
mediacondense.append(m6)

nwk = ['Newsweek magazine', 'Newsweek blog', 'Newsweek Magazine', 'Newsweek magazine blog']
mcat7 = "Newsweek"
m7 = dict.fromkeys(nwk, mcat7)
mediacondense.append(m7)

bbc = ['BBC Radio', 'BBC News blog', 'BBC Blogs', 'BBC News']
mcat8 = "BBC"
m8 = dict.fromkeys(bbc, mcat8)
mediacondense.append(m8)

fop = ['Foreign Policy Magazine May', 'Foreign Policy Journal']
mcat9 = "Foreign Policy"
m9 = dict.fromkeys(fop, mcat9)
mediacondense.append(m9)

wap = ['Washington Post blog', 'washingtonpost.com', 'Washingon Post', 'Washginton Post', 'The Washington Post']
mcat10 = "Washington Post"
m10 = dict.fromkeys(wap, mcat10)
mediacondense.append(m10)

tlg = ['The Telegraph blogs', 'Daily Telegraph', 'Telegraph']
mcat11 = "The Telegraph"
m11 = dict.fromkeys(tlg, mcat11)
mediacondense.append(m11)

nsa = ['U.S. National Security Agency Website', 'National Security Agency  Website', 'NSA Technical Journal, Vol. XI', 'National Security Agency  Website, NSA Technical Journal, Vol. XI']
mcat12 = "NSA Website"
m12 = dict.fromkeys(nsa, mcat12)
mediacondense.append(m12)

msn = ['MSN Money', 'MSN of Australia', 'MSN Canada', 'MSN']
mcat13 = "MSN News"
m13 = dict.fromkeys(msn, mcat13)
mediacondense.append(m13)

tim = ['Time magazine', 'Time Magazine', 'Time Magazine blog']
mcat14 = "Time"
m14 = dict.fromkeys(tim, mcat14)
mediacondense.append(m14)

psc = ['Popular Science - March 2007 Issue', 'Popular Science Magazine', 'Popular Science magazine']
mcat15 = "Popular Science"
m15 = dict.fromkeys(psc, mcat15)
mediacondense.append(m15)

cnn = ['CNN blog', 'CNN Money', 'CNN International', 'CNN The Situation Room', 'CNN Lou Dobbs Tonight', 'CNN Video Clip', 'CNN Larry King Live', 'CNN News']
mcat16 = "CNN"
m16 = dict.fromkeys(cnn, mcat16)
mediacondense.append(m16)

cbs = ['CBS Las Vegas Affiliate', 'CBS Philly', 'CBS Atlanta', 'CBS Affiliate KUTV', 'CBS News Chicago, Associated Press', 'CBS News, Sacramento Affiliate', 'WCBS News - New York CBS Affiliate', 'CBS News 60 Minutes', 'CBS News 60 Minutes Overtime', 'CBS Los Angeles', 'CBS 60 Minutes', 'CBS News blog', 'CBS News, Stockton Affiliate']
mcat17 = "CBS"
m17 = dict.fromkeys(cbs, mcat17)
mediacondense.append(m17)

yho = ['Yahoo! News', 'Yahoo!', 'Yahoo! Finance', 'Yahoo! News Australia', 'Yahoo Finance']
mcat18 = "Yahoo"
m18 = dict.fromkeys(yho, mcat18)
mediacondense.append(m18)

wsj = ['The Wall Street Journal', 'Wall Street Journal blog', 'Full Page Ad in Wall Street Journal', 'Wall Street Journal Article by Former FBI Director Louis Freeh', 'Wall Street Journal Blog']
mcat19 = "Wall Street Journal"
m19 = dict.fromkeys(wsj, mcat19)
mediacondense.append(m19)

fox = ['Fox News Chicago', 'Fox News video clip', 'FOX News', 'Fox 19', 'Fox News Affiliate']
mcat20 = "FOX"
m20 = dict.fromkeys(fox, mcat20)
mediacondense.append(m20)

icp = ['The Intercept With Glenn Greenwald', 'The Intercept with Glenn Greenwald']
mcat21 = "The Intercept"
m21 = dict.fromkeys(icp, mcat21)
mediacondense.append(m21)

lat = ['Los Angeles Times blog', 'The Los Angeles Times', 'LA Times']
mcat22 = "Los Angeles Times"
m22 = dict.fromkeys(lat, mcat22)
mediacondense.append(m22)

pbs = ['PBS Nova Program', 'PBS Frontline', 'PBS, CBS, Fox compilation', 'PBS News', 'PBS Bill Moyers Journal', 'PBS Newshour', 'PBS Blog']
mcat23 = "PBS"
m23 = dict.fromkeys(pbs, mcat23)
mediacondense.append(m23)

ecn = ['The Economist blog', 'The Economist Magazine', 'The Economist magazine']
mcat24 = "The Economist"
m24 = dict.fromkeys(ecn, mcat24)
mediacondense.append(m24)

npr = ['NPR All Things Considered', 'National Public Radio', 'NPR News', 'NPR blog', 'NPR Blog']
mcat25 = "MPR"
m25 = dict.fromkeys(npr, mcat25)
mediacondense.append(m25)

sfc = ['The San Francisco Chronicle', 'San Francisco Chronicle SFs leading newspaper)']
mcat26 = "San Francisco Chronicle"
m26 = dict.fromkeys(sfc, mcat26)
mediacondense.append(m26)

cbc = ['Canadian Broadcasting Corporation', 'CBC News', 'CBC News [Canadas Public Broadcasting System]']
mcat27 = "CBC"
m27 = dict.fromkeys(cbc, mcat27)
mediacondense.append(m27)
    
frb = ['Forbes Magazine', 'Forbes blog', 'Forbes magazine', 'Forbes.com', 'Forbes.com blog', 'Forbes India Magazine']
mcat28 = "Forbes"
m28 = dict.fromkeys(frb, mcat28)
mediacondense.append(m28)
    
rst = ['Rolling Stone blog', 'Rolling Stone magazine']
mcat29 = "Rolling Stone"
m29 = dict.fromkeys(rst, mcat29)
mediacondense.append(m29)
    
grd = ['A Guardian blog', 'The Guardian blog', 'Guardian']
mcat30 = "The Guardian"
m30 = dict.fromkeys(grd, mcat30)
mediacondense.append(m30)
    
ngc = ['NationalGeographic.com', 'National Geographic October 2004 Issue', 'National Geographic News', 'NationalGeographic.com blog']
mcat31 = "National Geographic"
m31 = dict.fromkeys(ngc, mcat31)
mediacondense.append(m31)
    
mbc = ['MSNBC News', 'MSNBC Today', 'MSNBC: Keith Olbermann blog', 'MSNBC The Rachel Maddow Show']
mcat32 = "MSNBC"
m32 = dict.fromkeys(mbc, mcat32)
mediacondense.append(m32)
    
rut = ['Reuters News Agency', 'Reuters News', 'Reuters Health']
mcat33 = "Reuters"
m33 = dict.fromkeys(rut, mcat33)
mediacondense.append(m33)
                                             

In [ ]:
#the list of dictionaries is then turned into a new little dataframe

key = []
val = []
for i in mediacondense:
    for k,v in i.items():
        key.append(k)
        val.append(v)

dfx = pd.DataFrame({'asis': key,
                  'clean': val})

#replacement values are mapped from new dataframe to complete main dataframe 
dfx.set_index('asis', inplace=True)
dx = pd.Series(dfx.clean.values,index=dfx.index).to_dict()
dfR['cpub'] = dfR['pub'].map(dx).fillna(dfR['pub'])

#create new dataframe with final column order, naming, and sort specification
dfF = pd.DataFrame(dfR[['ArticleId', 'tags', 'Title', 'quote', 'description', 'Links', 'cpub', 'PublicationDate', 'note', 'wtkURL', 'Priority']].sort_values(by='Priority', ascending=False))

dfF.drop(dfF.index[1000:], inplace=True)

#convert tags list column to kumu specification for export 
dfF['tags'] = dfF['tags'].astype(str).apply(lambda t: t.strip('['))
dfF['tags'] = dfF['tags'].astype(str).apply(lambda u: u.strip(']'))
dfF['tags'] = dfF['tags'].str.replace(',', '|')
dfF['tags'] = dfF['tags'].str.replace('\'', '')

dfF.columns = ['Label', 'Tags', 'Title', 'Quote', 'Description', 'SourceLink', 'Publisher', 'Date', 'Note', 'SummaryURL', 'Rating']
dfF.insert(loc=1, column='Type', value="Report", allow_duplicates=True)

#get labels to match to tags edge list
idsused = dfF['Label'].unique()

#dfF['Publisher'].value_counts()
dfF.shape

In [ ]:
#read in tag edges after confirming everything looks okay
taged = pd.read_csv("C:\\datasources\\tagedges.csv", sep='|', usecols=['ArticleId','tag'], encoding='utf-8')

#then create some correctly shaped element frames and append to main dataframe

list=[]
p = dfF['Publisher'].unique()
for i in p:
    list.append([i, "NewsSource", '','','','','','','','','',''])
dflist=pd.DataFrame(list,columns=['Label', 'Type', 'Tags', 'Title', 'Quote', 'Description', 'SourceLink', 'Publisher', 'Date', 'Note', 'SummaryURL', 'Rating'])

L2=[]
g = taged['tag'].unique()

for i in g:
    L2.append([i, "Topic", '','','','','','','','','',''])
dfL2=pd.DataFrame(L2,columns=['Label', 'Type', 'Tags', 'Title', 'Quote', 'Description', 'SourceLink', 'Publisher', 'Date', 'Note', 'SummaryURL', 'Rating'])

dfL2 = dfL2.append(dflist,ignore_index=True)
dfF[['Label', 'Rating']] = dfF[['Label', 'Rating']].astype('object', copy=False)
elements = pd.concat([dfF, dfL2],ignore_index=True)

#elements.tail()
#start making some edges
dfEdges = dfF[['Publisher', 'Label']]
dfEdges.columns=['From', 'To']
e2 = pd.DataFrame(taged.loc[taged['ArticleId'].isin(idsused)],columns=['ArticleId','tag'])
e2.columns=['From', 'To']
connections = pd.concat([dfEdges, e2],ignore_index=True)
connections.tail()

In [ ]:
writer = pd.ExcelWriter("C:\\datasources\\kumutestmap.xlsx")
elements.to_excel(writer,'Sheet1', index=False)
connections.to_excel(writer,'Sheet2', index=False)
writer.save()

In [ ]:
#misc bits for next steps

srch = dfR.groupby('ArticleId')['wtkURL'].apply(lambda t: list(t)).to_dict()
#find direct summary url by inserting ArticleId for XXXX in "print(srch[XXXX])"
print(srch[5250])

from sklearn import preprocessing
dfF['utc'] = pd.to_datetime(dfF['Date'], utc=True, unit='d')
# Create r and t to score rating and  column's values as floats
r = dfF[['Rating']].values.astype(float)
t = dfF[['utc']].values.astype(float)

# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
rX = min_max_scaler.fit_transform(r)
tX = min_max_scaler.fit_transform(t)

# Run the normalizer on the dataframe
df_normalized = pd.DataFrame(x_scaled)


#connections.sort_values(['ArticleId', 'cid'], ascending=[False, True]
#new = connections.sort_values(by='ArticleId', ascending=False)
#new[['ArticleId', 'tag']].head(20)
#new[new.ArticleId >= 9000]
